# FM 24 Moneyball

In [96]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show all columns in pandas
pd.set_option('display.max_columns', 500) 

%matplotlib inline

## Ingest and clean data

In [97]:
### Read raw data

# Select file path for raw data
path = r'data\Man Utd Save\Winter 2023\Players Search - Winter 2023.html'

# Ingest in a dataframe
df_players_raw = pd.read_html(path, encoding='utf-8')[0]

# Format column names 
df_players_raw.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players_raw.columns]

# ---------------------------------------------------------------------------------------------------------------------

### Read processed Data lab data

# Select file path for processed data
path = r'data\Man Utd Save\Winter 2023\Players Search - Winter 2023.csv'

# Ingest in a dataframe 
df_players = pd.read_csv(path)

# Format column names 
df_players.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players.columns]

# Bring ID from raw data
df_players = df_players.merge(df_players_raw[['name', 'age', 'height', 'weight', 'uid']], 
                 left_on=['name', 'age', 'height', 'weight'], 
                 right_on=['name', 'age', 'height', 'weight']).reset_index().rename(columns={'level_0': 'id'})

df_players['id'] = df_players['uid']

df_players.drop(columns='uid', inplace=True)

## Data cleaning

In [ ]:
# Create field with a list of positions 
#df_players['positions_list'] = df_players['position'].apply(lambda x: x.split(','))

In [100]:
df_players.head(10)

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,goals,goals/90,minutes/goal,xg,xg/90,xg/shot,xg_overperformance,xg_overperformance/90,non_pen_goals,non_pen_goals/90,non_pen_goals/shot,minutes/non_pen_goal,non_pen_xg,non_pen_xg/90,non_pen_goals___non_pens_xg_/90,non_pen_xg/shot,non_pen_xg_overperformance,non_pen_xg_overperformance/90,goals_outside_box,goals_outside_box/90,assists,assists/90,minutes/assist,xa,xa/90,xa_overperformance,xa_overperformance/90,assists/clear_cut_chances_created,goal_contributions,goal_contributions/90,xgc,xgc/90,xgc_overperformance,xgc_overperformance/90,non_pen_goal_contributions,non_pen_goal_contributions/90,non_pen_xgc,non_pen_xgc/90,non_pen_xgc_overperformance,non_pen_xgc_overperformance/90,conversion_%,shots,shots/90,shots_on_target,shots_on_target/90,shots_on_target_%,shots_outside_box/90,passes_attempted,passes_attempted/90,passes_completed,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_passes,progressive_pass_rate,key_passes,key_passes/90,key_pass_%,open_play_key_passes,open_play_key_passes/90,open_play_key_pass_%,crosses_attempted,crosses_attempted/90,crosses_completed,crosses_completed/90,crosses_completed_%,open_play_crosses_attempted,open_play_crosses_attempted/90,open_play_crosses_completed,open_play_crosses_completed/90,open_play_cross_completion_%,chances_created,chances_created/90,clear_cut_chances_created,clear_cut_chances_created/90,pressures_attempted,pressures_attempted/90,pressures_completed,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,dribbles/90,dribbles,penalties_taken,penalties_scored,pens_scored_%,tackles_attempted,tackles_attempted/90,tackles_completed,tackles_completed/90,tackles_failed,tackle_completion_%,tackles_failed/90,key_tackles,key_tackles/90,tackle_quality,interceptions,interceptions/90,blocks,blocks/90,shots_blocked,shots_blocked/90,headers_attempted,headers_attempted/90,headers_won,headers_won/90,headers_won_%,headers_lost,headers_lost/90,headers_lost_%,key_headers,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_against,fouls_made,net_fouls,fouls_won/90,fouls_committed/90,clearances,clearances/90,offsides,offsides/90,offside/non_pen_goals,offside/non_pen_xg,distance_covered/90,distance_covered,total_saves,saves/90,save_%,xsave_%,xsave_%_overperformance,saves_held,saves_held/90,saves_held_ratio,saves_held/shots_faced_ratio,saves_tipped,saves_tipped/90,saves_tipped_ratio,saves_tipped/shots_faced_ratio,saves_parried,saves_parried/90,saves_parried_ratio,saves_parried/shots_faced_ratio,saves/goal_conceaded,save_efficiency,shots_on_target_against,shots_on_target_against/90,xgp,xgp/90,penalties_faced,penalties_saved,pens_saved_%,goals_conceded,conceded/90,clean_sheets,clean_sheet_ratio,red_cards,yellow_cards,yellows/tackle,reds/tackle,yellows/90,reds/90,player_of_the_match,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,goalkeeping_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,positions_list
0,93070286,Alexander Isak,ST (C),24,190 cm,74 kg,-,Newcastle,Premier League,SWE (ERI),-,Fairly Ambitious,Media-friendly,"$150,000 p/w",$90M - $131M,-,Right,19,1602,6.84,2,76.29,0,7,0.39,228.86,6.52,0.37,0.22,0.48,0.03,6,0.34,0.20,267.00,5.76,0.32,0.02,0.19,0.24,0.01,1,0.06,3,0.17,534.0,2.09,0.12,0.91,0.05,0.0,10,0.56,8.61,0.48,1.39,0.08,9,0.51,7.85,0.44,1.15,0.06,23,30,1.69,17,0.96,57,0.39,317,17.81,262,14.72,83,1.12,20,6.31,10,0.56,3.15,10,0.56,3.15,20,1.12,1,0.06,5,20,1.12,1,0.06,5.00,8,0.43,4,0.22,119,6.69,39,2.19,32.77,3.82,6.74,-2.92,-43.32,1.35,

## Goalkeepers

## Center Backs

In [101]:
# Filter players that can play center back 
# We are filtering D(C), D(CL)
df_cb = df_players[df_players['position'].str.contains(r'D \([A-Z]*C[A-Z]*\)', regex=True, na=False)]

In [104]:
df_cb

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,goals,goals/90,minutes/goal,xg,xg/90,xg/shot,xg_overperformance,xg_overperformance/90,non_pen_goals,non_pen_goals/90,non_pen_goals/shot,minutes/non_pen_goal,non_pen_xg,non_pen_xg/90,non_pen_goals___non_pens_xg_/90,non_pen_xg/shot,non_pen_xg_overperformance,non_pen_xg_overperformance/90,goals_outside_box,goals_outside_box/90,assists,assists/90,minutes/assist,xa,xa/90,xa_overperformance,xa_overperformance/90,assists/clear_cut_chances_created,goal_contributions,goal_contributions/90,xgc,xgc/90,xgc_overperformance,xgc_overperformance/90,non_pen_goal_contributions,non_pen_goal_contributions/90,non_pen_xgc,non_pen_xgc/90,non_pen_xgc_overperformance,non_pen_xgc_overperformance/90,conversion_%,shots,shots/90,shots_on_target,shots_on_target/90,shots_on_target_%,shots_outside_box/90,passes_attempted,passes_attempted/90,passes_completed,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_passes,progressive_pass_rate,key_passes,key_passes/90,key_pass_%,open_play_key_passes,open_play_key_passes/90,open_play_key_pass_%,crosses_attempted,crosses_attempted/90,crosses_completed,crosses_completed/90,crosses_completed_%,open_play_crosses_attempted,open_play_crosses_attempted/90,open_play_crosses_completed,open_play_crosses_completed/90,open_play_cross_completion_%,chances_created,chances_created/90,clear_cut_chances_created,clear_cut_chances_created/90,pressures_attempted,pressures_attempted/90,pressures_completed,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,dribbles/90,dribbles,penalties_taken,penalties_scored,pens_scored_%,tackles_attempted,tackles_attempted/90,tackles_completed,tackles_completed/90,tackles_failed,tackle_completion_%,tackles_failed/90,key_tackles,key_tackles/90,tackle_quality,interceptions,interceptions/90,blocks,blocks/90,shots_blocked,shots_blocked/90,headers_attempted,headers_attempted/90,headers_won,headers_won/90,headers_won_%,headers_lost,headers_lost/90,headers_lost_%,key_headers,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_against,fouls_made,net_fouls,fouls_won/90,fouls_committed/90,clearances,clearances/90,offsides,offsides/90,offside/non_pen_goals,offside/non_pen_xg,distance_covered/90,distance_covered,total_saves,saves/90,save_%,xsave_%,xsave_%_overperformance,saves_held,saves_held/90,saves_held_ratio,saves_held/shots_faced_ratio,saves_tipped,saves_tipped/90,saves_tipped_ratio,saves_tipped/shots_faced_ratio,saves_parried,saves_parried/90,saves_parried_ratio,saves_parried/shots_faced_ratio,saves/goal_conceaded,save_efficiency,shots_on_target_against,shots_on_target_against/90,xgp,xgp/90,penalties_faced,penalties_saved,pens_saved_%,goals_conceded,conceded/90,clean_sheets,clean_sheet_ratio,red_cards,yellow_cards,yellows/tackle,reds/tackle,yellows/90,reds/90,player_of_the_match,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,goalkeeping_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,positions_list
1371,37060952,Quinten Timber,"D (C), DM, M/AM (C)",22,177 cm,71 kg,-,Feyenoord,Eredivisie,NED (CUW),-,Spirited,"Evasive, Reserved","$18,250 p/w",$53M - $77M,-,Right,15,1325,7.05,3,73.61,1371,1,0.07,1325.00,1.80,0.12,0.09,-0.80,-0.05,1,0.07,0.05,1325.00,1.80,0.12,-0.05,0.09,-0.80,-0.05,0,0.00,4,0.27,331.25,2.85,0.19,1.15,0.08,0.80,5,0.34,4.65,0.32,0.35,0.02,5,0.34,4.65,0.32,0.35,0.02,5,19,1.29,7,0.48,37,0.75,934,63.44,821,55.77,88,7.00,103,11.03,33,2.24,3.53,33,2.24,3.53,12,0.82,3,0.20,37,8,0.54,3,0.20,37.50,11,0.75,5,0.34,145,9.85,29,1.97,20.00,10.05,

## Full Backs